In [2]:
import pandas as pd

ANALYSIS ON IDS

In [8]:
#import IDS data
IDS_data = pd.read_csv(r'C:\Users\user\Desktop\MOPA Reconciliation\Bugesera\Season client\Season Clients Detailed_20200926-021956.csv')


In [79]:
#Preliminary info on IDS data on subsidized inputs
GovLocationParent = 'Shyara'
SectorDf = IDS_data [IDS_data ['GovLocationParent']==GovLocationParent ]
SectorClients = SectorDf.groupby('GovLocationParent')['OAFID'].count()

SubsInputsdf = SectorDf[['GovLocationGrandParent','GovLocationParent','SiteName','LastName','FirstName','OAFID','NationalID','2021A_DAP kg','2021A_NPK 17 kg','2021A_Pan 53 kg','2021A_Pan 691 kg','2021A_SC 403 kg','2021A_SC 637 kg','2021A_UREA kg','2021A_WH 403 kg','2021A_WH 505 kg','2021A_WH 605 kg']]
SubsInputsdf['NationalID'] =SubsInputsdf['NationalID'].astype(str).replace('\.0', '', regex=True)

SubsInputsMelt = SubsInputsdf.melt(['GovLocationGrandParent','GovLocationParent','SiteName','LastName','FirstName','OAFID','NationalID'],['2021A_DAP kg','2021A_NPK 17 kg','2021A_Pan 53 kg','2021A_Pan 691 kg','2021A_SC 403 kg','2021A_SC 637 kg','2021A_UREA kg','2021A_WH 403 kg','2021A_WH 505 kg','2021A_WH 605 kg'], 'inputName','Quantity').fillna(0)

#Cleaning some IDS columns
SubsInputsMelt['NationalID'] =SubsInputsMelt['NationalID'].astype(str).replace('\.0', '', regex=True)

IDSInputsTxns = SubsInputsMelt
IDSInputsTxns ['inputName'] = IDSInputsTxns ['inputName'].replace(['2021A_DAP kg','2021A_NPK 17 kg','2021A_UREA kg','2021A_WH 403 kg','2021A_SC 403 kg','2021A_Pan 53 kg'],['DAP','NPK','UREA','WH403','SC403','PAN53'])
IDSInputsTxns =IDSInputsTxns.rename(columns={'inputName':'Product'})
IDSInputsTxns['ProductID'] = IDSInputsTxns['NationalID'] + IDSInputsTxns['Product']

#Check if NID starts with 119
SubsInputsdf['NIDStartsWith119'] = SubsInputsdf['NationalID'].str.startswith('119')
NIDStartsWith119 = SubsInputsdf[SubsInputsdf['NIDStartsWith119']==False]
NIDStartsWith119_site = NIDStartsWith119.groupby('SiteName')['OAFID'].count().fillna(0)

#Check if NID contains either 700 for female and 800 for male
SubsInputsdf['FemaleCheck'] = SubsInputsdf['NationalID'].str.contains(pat = '700')
SubsInputsdf['maleCheck'] = SubsInputsdf['NationalID'].str.contains(pat = '800')
SubsInputsdf['FalseGender'] = SubsInputsdf['FemaleCheck']== SubsInputsdf['maleCheck']
FalseGender = SubsInputsdf[SubsInputsdf['FalseGender']==True]
FalseGender_site = FalseGender.groupby('SiteName')['OAFID'].count().fillna(0)

#Check if NID length is equal to 16
SubsInputsdf['NIDLength'] = SubsInputsdf['NationalID'].str.len()==16
NIDLength = SubsInputsdf[SubsInputsdf['NIDLength']==False]
NIDLength_site = NIDLength.groupby('SiteName')['OAFID'].count().fillna(0)

#FINAL NID FRAME
CheckOnNID = pd.merge(pd.merge(NIDStartsWith119_site,FalseGender_site, on='SiteName', how='outer'),NIDLength_site, on='SiteName',how='outer').fillna(0).reset_index()
CheckOnNID['#Wrong NID'] = CheckOnNID['OAFID_x']+CheckOnNID['OAFID_y']+CheckOnNID['OAFID']

#IDSClientsPerInput = IDSInputsTxns.groupby('Product')['OAFID'].count().reset_index()
#IDSQtyPerInput = IDSInputsTxns.groupby('Product')['Quantity'].sum().reset_index()
#SectorSummary = pd.merge(IDSClientsPerInput,IDSQtyPerInput,on='Product',how='outer').rename(columns={'OAFID':'#Clients'})
#SectorSummary 

#(1) FINDING THE TOTAL NUMBER OF CLIENTS IN EACH SITE ON IDS
TotalSiteClients = SectorDf.groupby('SiteName')['OAFID'].count().reset_index()

#(2)FINDING THE TOTAL NUMBER OF WRONG NID
ClientsWithWrongNID = CheckOnNID[['SiteName','#Wrong NID']]

#(3)FINDING THE TOTAL QUANTITY ON IDS IN EACH SITE
TotalKgsPerSite = pd.pivot_table(IDSInputsTxns, values='Quantity', index=['GovLocationGrandParent', 'GovLocationParent', 'SiteName'],columns=['Product'], aggfunc='sum').reset_index()

#(4)TOTAL NUMBER OF CLIENTS WHO TOOK SPECIFIC PRODUCTS
ClientsPerInputs = pd.pivot_table(IDSInputsTxns, values='NationalID', index=['GovLocationGrandParent', 'GovLocationParent', 'SiteName'],columns=['Product'], aggfunc='count').reset_index()

#(5)MATCHING THE NUMBER OF CLIENTS AND TOTAL KGS TAKEN
IDS = pd.merge(pd.merge(ClientsPerInputs,TotalKgsPerSite, on='SiteName', how='outer' ),ClientsWithWrongNID, on='SiteName', how='outer')

In [80]:
IDS

,GovLocationGrandParent_x,GovLocationParent_x,SiteName,DAP_x,NPK_x,SC403_x,UREA_x,GovLocationGrandParent_y,GovLocationParent_y,DAP_y,NPK_y,SC403_y,UREA_y,#Wrong NID
0,Bugesera,Shyara,Kabagugu,362,48,354,348,Bugesera,Shyara,3420.0,775.0,914.0,2810.0,174.0
1,Bugesera,Shyara,Kamabuye,306,22,325,276,Bugesera,Shyara,2555.0,215.0,792.0,1985.0,163.0
2,Bugesera,Shyara,Nziranziza,493,58,460,482,Bugesera,Shyara,7360.0,1350.0,1077.0,4305.0,282.0
3,Bugesera,Shyara,Rebero,456,80,449,456,Bugesera,Shyara,5070.0,1475.0,1168.0,4655.0,246.0
4,Bugesera,Shyara,Rutare,472,92,472,418,Bugesera,Shyara,4700.0,1005.0,1206.0,3405.0,214.0


ANALYSIS ON MOPA

In [82]:
#import MOPA data
MOPA = pd.read_excel(r'C:\Users\user\Desktop\MOPA Reconciliation\Bugesera\MOPA data\Shyara.xlsx')

In [83]:
#Cleaning MOPA data
ColSplit = MOPA ['Requested Qty'].str.split(" ", n = 2, expand = True)
MOPA['Requested_qty'] = ColSplit[0].astype(float)
MOPA['Paid_qty'] = MOPA['Paid Qty'].str.extract('(\d+)').astype(float)
MOPA['Product'] = MOPA['Product'].str.upper()
MOPA = MOPA.rename(columns={'National Id':'NationalID'})
MOPA['NationalID'] = MOPA['NationalID'].astype(str)
MOPA['ProductID'] = MOPA['NationalID'] + MOPA['Product']
MOPA_dataCols = MOPA[['Name','NationalID','Product','Requested_qty','Paid_qty']]

In [87]:
MOPA_Sites = pd.merge(IDSInputsTxns, MOPA, on='ProductID',how='outer')

MOPA_Sites.groupby('SiteName')[]

,GovLocationGrandParent,GovLocationParent,SiteName,LastName,FirstName,OAFID,NationalID_x,Product_x,Quantity,ProductID,...,Name,Surname,NationalID_y,Product_y,Requested Qty,Paid Qty,Payment Method,Unpaid Qty,Requested_qty,Paid_qty
0,Bugesera,Shyara,Kabagugu,Fidel,Bizimungu,-1.025000e+09,1198080156846063,DAP,10.0,1198080156846063DAP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Bugesera,Shyara,Kabagugu,Reverien,Semandwa,-1.025000e+09,1199180125066055,DAP,5.0,1199180125066055DAP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Bugesera,Shyara,Kabagugu,Aloys,Ndayisaba,-1.025000e+09,1197780037399011,DAP,5.0,1197780037399011DAP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Bugesera,Shyara,Kabagugu,Beatrice,Mukabutera,-1.025000e+09,1197370070640008,DAP,5.0,1197370070640008DAP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Bugesera,Shyara,Kabagugu,Dativa,Mukeshimana,-1.025000e+09,1199970038754051,DAP,5.0,1199970038754051DAP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
#Preliminary info on IDS data on subsidized inputs

ClientsqtyPerInput =MOPA_dataCols.groupby('Product')['Requested_qty','Paid_qty'].sum().reset_index()
ClientsNumPerInput = MOPA_dataCols.groupby('Product')['NationalID'].count().reset_index()

#Number of clients who took each input in mopa and the total quantity taken per each input
SummaryMOPA = pd.merge(ClientsqtyPerInput,ClientsNumPerInput,on='Product', how='outer').rename(columns={'NationalID':'#Clients'})
SummaryMOPA['Product'] = SummaryMOPA['Product'].replace(['NPK 17:17:17'],['NPK'])
SummaryMOPA

,Product,Requested_qty,Paid_qty,#Clients
0,DAP,52997.0,20342.0,980
1,NPK,1990.0,570.0,19
2,SC403,314.5,88.0,29
3,UREA,39882.0,8966.0,792


In [51]:
#Other data on IDS
TotalMOPAClients = MOPA['NationalID'].nunique()
#Transactions
TotalMOPATxn = MOPA['NationalID'].count()

MATCHING IDS TO MOPA 

In [52]:
 MOPAvsIDS = pd.merge(SectorSummary,SummaryMOPA,on='Product',how='outer')
 MOPAvsIDS['Difference In #Clients'] = MOPAvsIDS['#Clients_x'] - MOPAvsIDS['#Clients_y']
 MOPAvsIDS['Difference In qty'] = MOPAvsIDS['Quantity'] - MOPAvsIDS['Paid_qty']
 MOPAvsIDS

,Product,#Clients_x,Quantity,Requested_qty,Paid_qty,#Clients_y,Difference In #Clients,Difference In qty
0,DAP,2089,23105.0,52997.0,20342.0,980,1109,2763.0
1,NPK,300,4820.0,1990.0,570.0,19,281,4250.0
2,SC403,2060,5157.0,314.5,88.0,29,2031,5069.0
3,UREA,1980,17160.0,39882.0,8966.0,792,1188,8194.0


CLIENT LEVEL CHECK

In [53]:
IDSInputsTxns = IDSInputsTxns.rename(columns={'NationalID':'IDS_NationalID','Product':'IDS_Product','Quantity':'IDS_Quantity','GovLocationParent':'GovSector','GovLocationGrandParent':'GovDistrict'})
IDSInputsTxns['SiteID'] = IDSInputsTxns['GovSector']+IDSInputsTxns['SiteName']
MOPAInputsTxns = MOPA[['Name', 'Surname', 'NationalID','Product','Paid_qty','ProductID']]
MOPAInputsTxns = MOPAInputsTxns.rename(columns={'NationalID':'SNS_NationalID','Product':'SNS_Product','Paid_qty':'SNS_Quantity'})
IDSvsMOPA_Clients = pd.merge(IDSInputsTxns,MOPAInputsTxns,on='ProductID', how='outer')
IDSvsMOPA_Clients['Match'] = IDSvsMOPA_Clients['IDS_Quantity'] == IDSvsMOPA_Clients['SNS_Quantity']


IDSvsMOPA_Clients['IDS_NationalID'] = "'"+ IDSvsMOPA_Clients['IDS_NationalID']
IDSvsMOPA_Clients['SNS_NationalID'] = "'"+ IDSvsMOPA_Clients['SNS_NationalID']
path = r'C:\Users\user\Desktop\MOPA Reconciliation\Bugesera\MOPAvsIDS data'
os.chdir(path)
IDSvsMOPA_Clients.to_csv('Shyara.csv', index=False)

In [34]:
#clients
TotalClientsOnMOPA_IDS = IDSvsMOPA_Clients['OAFID'].nunique()

#Transactions
MatchingTxn = IDSvsMOPA_Clients[IDSvsMOPA_Clients['Match']==True]['OAFID'].count()
NotMatchingTXn = IDSvsMOPA_Clients[IDSvsMOPA_Clients['Match']==False]['OAFID'].count()
OnIDSNotInMOPA = IDSvsMOPA_Clients[IDSvsMOPA_Clients['Paid_qty'].isna()]

KeyError: 'Paid_qty'

,DistrictName,SectorName,SiteName,LastName,FirstName,OAFID,NationalID_x,Product_x,Quantity,ProductID,Name,Surname,NationalID_y,Product_y,Paid_qty,Match
0,Nyanza,Zone 1,Butansinda,URINZWENIMANA,Vianney,79768,1197680097920090,DAP,5.0,1197680097920090DAP,NaN,NaN,NaN,NaN,NaN,False
1,Nyanza,Zone 1,Ngwa,Ahobantegeye,Cecile,69895,1196570014701050,DAP,10.0,1196570014701050DAP,NaN,NaN,NaN,NaN,NaN,False
2,Nyanza,Zone 1,Ngwa,DUSABEMARIYA,Faranswaze,29024,1198870015488075,DAP,10.0,1198870015488075DAP,NaN,NaN,NaN,NaN,NaN,False
3,Nyanza,Zone 1,Ngwa,Kanyanja,Athanase,69896,1194270004772161,DAP,10.0,1194270004772161DAP,NaN,NaN,NaN,NaN,NaN,False
4,Nyanza,Zone 1,Ngwa,Mukarurinda,Hana,20833,1197170016487095,DAP,5.0,1197170016487095DAP,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6102,Nyanza,Zone 7,Nkomero,Mushimiyimana,Catheline,27094,1196570014066166,UREA,5.0,1196570014066166UREA,MUSHIMIYIMANA,GATARINA,1196570014066166,UREA,5.0,True
6103,Nyanza,Zone 7,Nkomero,Rucogoza,Aimable,73239,1197880036856041,UREA,5.0,1197880036856041UREA,RUCOGOZA,AIMABLE,1197880036856041,UREA,5.0,True
6104,Nyanza,Zone 7,Nkomero,Seromba,Yasoni,37554,1197980033140032,UREA,5.0,1197980033140032UREA,NaN,NaN,NaN,NaN,NaN,False
6105,Nyanza,Zone 7,Nkomero,UKOBUKEYE,Eugene,25009,1196480014386022,UREA,5.0,1196480014386022UREA,NaN,NaN,NaN,NaN,NaN,False


In [14]:
#Print clients
print('Total clients on IDS:',TotalIDSClients)
print('Total clients in MOPA:',TotalMOPAClients)
print('Total clients on IDS_MOPA df:',TotalClientsOnMOPA_IDS )


#Transactions
print('Total Txn on IDS:',IDSTotalTxn)
print('Total Txn in MOPA:',TotalMOPATxn)
print('Total Txn IDS match MOPA:',MatchingTxn)
print('Total Txn IDS dont match MOPA:',NotMatchingTXn)

Total clients on IDS: 2191
Total clients in MOPA: 1743
Total clients on IDS_MOPA df: 2079
Total Txn on IDS: 6107
Total Txn in MOPA: 3666
Total Txn IDS match MOPA: 2147
Total Txn IDS dont match MOPA: 3960


In [19]:
NIDCheck = IDSvsMOPA_Clients[['LastName','FirstName','OAFID','NationalID_x']]
NIDCheck['NIDStartsWith'] = NIDCheck['NationalID_x'].str.startswith('119')
NIDCheck['FemaleCheck'] = NIDCheck['NationalID_x'].str.contains(pat = '700')
NIDCheck['maleCheck'] = NIDCheck['NationalID_x'].str.contains(pat = '800')
NIDCheck['GenderCheck'] = NIDCheck['FemaleCheck']== NIDCheck['maleCheck']

,LastName,FirstName,OAFID,NationalID_x,NIDStartsWith,FemaleCheck,maleCheck,GenderCheck
6,Niyonsaba,Aloys,20825,1199080221005077,True,False,False,True
21,UWAMURENGEYE,Japhet,77763,1198080141925048,True,False,False,True
46,Mukamana,Anastasia,27069,1199480191081057,True,False,False,True
57,TWAHIRWA,Osuard,61821,1198080168232064,True,False,False,True
68,Ndayisenga,Furaha,62485,1198870036778004,True,True,True,True
...,...,...,...,...,...,...,...,...
6043,Nsanzabera,Tito,60321,1199480125127064,True,False,False,True
6054,MUKANKUSI,Pelagie,71162,0,False,False,False,True
6056,NGAYABARAMBIRWA,Gadi,71171,0,False,False,False,True
6066,Mutijima,Joni,62490,1198380163945071,True,False,False,True
